# Model serving & deployment testing

## Abstract

This notebook test model deployment using a pre-transformed example.

In [109]:
ENDPOINT_ID='9117141621425569792' 
GOOGLE_CLOUD_PROJECT = 'aliz-ml-spec-2022-dev'
GOOGLE_CLOUD_REGION = 'us-central1'

In [ ]:
!gsutil cp -r gs://mormota/pipeline_root/taxi-vertex-pipelines/639006805448/taxi-vertex-pipelines-20220608184748/Transform_-2682517550947893248/transformed_examples transform/

In [ ]:
!gunzip transform/transformed_examples/Split-eval/transformed_examples-00000-of-00001.gz

In [110]:
import numpy as np
from google.cloud import aiplatform
import tensorflow as tf

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)


raw_dataset = tf.data.TFRecordDataset("transform/transformed_examples/Split-eval/transformed_examples-00000-of-00001")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

instance_dict = {}
for feature_name in example.features.ListFields()[0][1]:
    if feature_name != 'fare':
        try:
            instance_dict[feature_name] = [example.features.ListFields()[0][1][feature_name].float_list.value[0]]
        except:
            instance_dict[feature_name] = [example.features.ListFields()[0][1][feature_name].int64_list.value[0]]

instances = [instance_dict]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
response = client.predict(endpoint=endpoint, instances=instances)
print('prediction:', response.predictions[0])

prediction: [5.70512199]


The prediction of 5.7 we get back is fairly close to the expected 8 from the example